<a href="https://colab.research.google.com/github/Danny-Jang/naver_movie_review_sentiment_analysis/blob/master/Mecab_kor_embedding_vector_FastText_%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

## install konlpy library

In [0]:
!pip install konlpy


## install jdk and install JPype1-py3

oracle jdk 설치 무방

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip3 install JPype1-py3



## mecab-ko 설치
[mecab 설치 문서](https://bitbucket.org/eunjeon/mecab-ko)

In [0]:
import os
os.chdir('/tmp/')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.1.tar.gz
!tar zxfv mecab-0.996-ko-0.9.1.tar.gz
os.chdir('/tmp/mecab-0.996-ko-0.9.1')
!./configure
!make
!make check
!make install



## automake 설치

In [0]:

os.chdir('/tmp')
!curl -LO http://ftpmirror.gnu.org/automake/automake-1.11.tar.gz
!tar -zxvf automake-1.11.tar.gz
os.chdir('/tmp/automake-1.11')
!./configure
!make
!make install


    Please make sure it is installed and in your PATH.
make: *** No targets specified and no makefile found.  Stop.
make: *** No rule to make target 'install'.  Stop. 
위 문구가 나오면 실패 
automake install error를 실행



make[2]: Leaving directory '/tmp/automake-1.11/tests'
make[1]: Leaving directory '/tmp/automake-1.11/tests'
이 나오면 성공

## automake install error
configure: error: no acceptable m4 could be found in $PATH.

Glibc 2.9 - 2.12 and GNU M4 1.4.11 - 1.4.15 have another strstr bug.

- install GNU M4 1.4.9

- install autoconf-2.69



In [0]:
import os
os.chdir('/tmp/')

 
!wget -O m4-1.4.9.tar.gz http://ftp.gnu.org/gnu/m4/m4-1.4.9.tar.gz
!tar -zvxf m4-1.4.9.tar.gz
os.chdir('/tmp/m4-1.4.9')
!./configure
!make
!make install

os.chdir('/tmp')
!curl -OL http://ftpmirror.gnu.org/autoconf/autoconf-2.69.tar.gz
!tar xzf autoconf-2.69.tar.gz
os.chdir('/tmp/autoconf-2.69')
!./configure --prefix=/usr/local
!make
!make install
!export PATH=/usr/local/bin



## install mecab-ko-dic
[mecab-ko-dic 설치 문서](https://bitbucket.org/eunjeon/mecab-ko-dic)

In [0]:
import os
os.chdir('/tmp')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.0.1-20150920.tar.gz
!tar -zxvf mecab-ko-dic-2.0.1-20150920.tar.gz
os.chdir('/tmp/mecab-ko-dic-2.0.1-20150920')
!./autogen.sh
!./configure
!make
# !sh -c 'echo "dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic" > /usr/local/etc/mecabrc'
!make install



make: *** No targets specified and no makefile found.  Stop.

make: *** No rule to make target 'install'.  Stop.

Idconfig 를 실행

done!
echo To enable dictionary, rewrite /usr/local/etc/mecabrc as \"dicdir = /usr/local/lib/mecab/dic/mecab-ko-dic\"

위 문구가 나오면 성공 install Mecab python을 실행

## ldconfig
libmecab.so.2를 찾을 수 없는 에러가 나는 경우

실행 후 재시도 위로 올라가서 전체를 재시도 



In [0]:
os.chdir('/tmp/mecab-ko-dic-2.0.1-20150920')
!ldconfig
!ldconfig -p | grep /usr/local/lib


## install mecab-python
python3 기준입니다.

In [0]:
# install mecab-python
import os
os.chdir('/content')

!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
os.chdir('/content/mecab-python-0.996')

!python3 setup.py build
!python3 setup.py install


running install_egg_info

Writing /usr/local/lib/python3.6/dist-packages/mecab_python-0.996_ko_0.9.0.egg-info

위 문구가 나오면 Mecab 설치 완료 

In [0]:
from konlpy.tag import Okt, Komoran, Hannanum, Kkma , Mecab



okt = Okt()
komoran = Komoran()
hannanum = Hannanum()
kkma = Kkma()
mecab = Mecab()


tagger = Mecab()
tagger = tagger.morphs

In [0]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)

DOCUMENT = data.Field(tokenize = tagger ,include_lengths = False)
LABEL = data.LabelField(dtype = torch.float)


In [0]:
fields = {'document':('document',DOCUMENT),'label':('label', LABEL) }
# fields = [('id', None),('document',DOCUMENT),('label', LABEL) ]  skip_header = True

In [20]:
train_data, test_data = data.TabularDataset.splits(
                            path = '/content/gdrive/My Drive/naver_movie_review',  
                            train = "ratings_train.txt",
                            test =  "ratings_test.txt",
                            format = 'tsv',
                            fields = fields,
                            #skip_header = True
                            
)

print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data[2]))

Number of training examples: 150000
Number of testing examples: 50000
{'document': ['너무', '재', '밓었', '다', '그래서', '보', '는', '것', '을', '추천', '한다'], 'label': '0'}


In [21]:
print(okt.morphs('okt: 너무재밓었다그래서보는것을추천한다'))
print(komoran.morphs('komoran: 너무재밓었다그래서보는것을추천한다'))
print(hannanum.morphs('hannanum: 너무재밓었다그래서보는것을추천한다'))
print(kkma.morphs('kkma: 너무재밓었다그래서보는것을추천한다'))
print(mecab.morphs('mecab: 너무재밓었다그래서보는것을추천한다'))


['okt', ':', '너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다']
['komoran', ':', '너무재밓었다그래서보는것을추천한다']
['hannanum', ':', '너무재밓었다그래서보는것을추천한다']
['kkma', ':', '너무', '재', '밓', '어', '었', '다', '그래서', '보', '는', '것', '을', '추천', '하', 'ㄴ다']
['mecab', ':', '너무', '재', '밓었', '다', '그래서', '보', '는', '것', '을', '추천', '한다']


In [22]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))
print(len(train_data))
print(len(valid_data))
print(len(test_data))

105000
45000
50000


In [0]:
import torch.nn as nn
import torchtext.data as data

from torchtext.vocab import Vectors


MAX_VOCAB_SIZE = 45000

ko_vectors = Vectors(name ='ko.vec',cache ='/content/gdrive/My Drive/word2vec')

DOCUMENT.build_vocab(train_data, 
                     max_size = MAX_VOCAB_SIZE,
                     vectors = ko_vectors,                  
                     unk_init = torch.Tensor.normal_
                     )

LABEL.build_vocab(train_data)

In [24]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fac12c1d158>, {'0': 0, '1': 1})


In [0]:
BATCH_SIZE = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator,valid_iterator,test_iterator = data.BucketIterator.splits((train_data,valid_data,test_data),
                                                                                   batch_size = BATCH_SIZE,
                                                                                   sort = False,
                                                                                   device = device
                                                                                   )



In [0]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        

        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

In [27]:
INPUT_DIM = len(DOCUMENT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 512
OUTPUT_DIM = 1
PAD_IDX = DOCUMENT.vocab.stoi[DOCUMENT.pad_token]

print(INPUT_DIM)
print(OUTPUT_DIM)

45002
1


In [0]:
model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)
model = model.to(device)


In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
  """
  returns accuracy per batch, i.e, if you get 8/10 right, this returns 0.8, NOT 8

  """

  #round predictions to the closet interger
  rounded_preds = torch.round(torch.sigmoid(preds))
  correct = (rounded_preds == y).float() #conver into float for division
  acc = correct.sum()/ len(correct)
  return acc

In [0]:
def train(model, iterator, optimizer,criterion):

   epoch_loss = 0
   epoch_acc = 0 
   model.train()
   
  
  
 
   for batch in iterator:
       optimizer.zero_grad()
       predictions = model(batch.document).squeeze(1)
       loss = criterion(predictions, batch.label)
       acc = binary_accuracy(predictions, batch.label)
       loss.backward()
       optimizer.step()

       epoch_loss += loss.item()
       epoch_acc += acc.item()

   return epoch_loss / len(iterator), epoch_acc/len(iterator)

In [0]:
def evaluate(model,iterator, criterion):

  epoch_loss = 0
  epoch_acc = 0 
  model.eval()

  with torch.no_grad():
    for batch in iterator:
      

      predictions = model(batch.document).squeeze(1)
      loss = criterion(predictions,batch.label)
      acc = binary_accuracy(predictions, batch.label)

      epoch_loss += loss.item()
      epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc/len(iterator)

In [0]:
import time 

def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time/60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [34]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
         best_valid_loss = valid_loss
         torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 18s
	Train Loss: 0.495 | Train Acc: 77.93%
	 Val. Loss: 0.402 |  Val. Acc: 82.68%
Epoch: 02 | Epoch Time: 0m 18s
	Train Loss: 0.365 | Train Acc: 85.02%
	 Val. Loss: 0.372 |  Val. Acc: 84.34%
Epoch: 03 | Epoch Time: 0m 18s
	Train Loss: 0.331 | Train Acc: 86.71%
	 Val. Loss: 0.364 |  Val. Acc: 84.76%
Epoch: 04 | Epoch Time: 0m 18s
	Train Loss: 0.309 | Train Acc: 87.75%
	 Val. Loss: 0.364 |  Val. Acc: 84.90%
Epoch: 05 | Epoch Time: 0m 18s
	Train Loss: 0.293 | Train Acc: 88.50%
	 Val. Loss: 0.370 |  Val. Acc: 84.84%


In [35]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc =evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.369 | Test Acc: 84.74%
